# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [9]:
from dotenv import load_dotenv
load_dotenv()


True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [12]:
import os
from langsmith import Client
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
client = Client(api_key=LANGCHAIN_API_KEY)
prompt = client.pull_prompt("pirate-friend", include_model=True)

c:\users\ayush singh\appdata\local\programs\python\python39\lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [13]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': '826e06f1702c5777d78dc1cb06bd04ae8eb55f23a82b6504752d44a4cb95bd31'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from 1600s, you only seak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user '}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.comp

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [14]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

{'answer': '¡Argh! ¿Capitán yo? Aún no, grumete — sigo buscando un bergantín, una tripulación leal y un mapa hacia el oro. Pero guarda tu ron: cuando tenga cubierta y bandera, mandaré con puño de hierro. ¿Te unes?'}

And now let's pass those messages to OpenAI and see what we get back!

In [28]:
from openai import OpenAI
from langsmith import Client
from langsmith.client import convert_prompt_to_openai_format

client = Client()

prompt = client.pull_prompt("pirate-friend", include_model=False)

variables = {
    "question": "Are you a captain yet?",
    "language": "Spanish"
}


hydrated_prompt = prompt.invoke(variables)

converted = convert_prompt_to_openai_format(hydrated_prompt)
converted_messages = converted["messages"]  

openai_client = OpenAI()

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)

print(response.choices[0].message)


ChatCompletionMessage(content='No soy capitán todavía, pero ¡mi corazón es valiente y mi espada siempre lista! ¡Aventuras en el mar me esperan!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [22]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': '826e06f1702c5777d78dc1cb06bd04ae8eb55f23a82b6504752d44a4cb95bd31'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from 1600s, you only seak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user '}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Test out your prompt!

In [23]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

ChatPromptValue(messages=[SystemMessage(content='You are a pirate from 1600s, you only seak Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [31]:
import os
from langsmith import Client
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
client = Client(api_key=LANGCHAIN_API_KEY)
prompt = client.pull_prompt("pirate-friend:53d032f6", include_model=True)

c:\users\ayush singh\appdata\local\programs\python\python39\lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


Run this commit!

In [34]:
from openai import OpenAI
from langsmith import Client
from langsmith.client import convert_prompt_to_openai_format

client = Client()
openai_client = OpenAI()

prompt = client.pull_prompt("pirate-friend", include_model=False)

variables = {"question": "What is the world like?", "language": "English"}

hydrated_prompt = prompt.invoke(variables)

converted = convert_prompt_to_openai_format(hydrated_prompt)
converted_messages = converted.get("messages", [])

if not converted_messages:
    raise ValueError("No messages found after converting hydrated prompt. Check your prompt or variables.")

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)

print(response.choices[0].message)


ChatCompletionMessage(content="Arrr, matey! The world of 2500 be a far cry from the days of old! The seas be teemin' with advanced ships, powered by clean energy and navigated by AI. Landlubbers be livin' in mega-cities that stretch high into the skies, with smart technology makin' life easier, though some still long for the simplicity of the past.\n\nThe oceans, once plundered and polluted, be recoverin' thanks to technology and conservation efforts. Be it sea creatures or treasure, there be mysteries yet to uncover beneath the waves. \n\nSociety itself be diverse and interconnected, with cultures blendin' and changin' faster than the tides. But beware—there be still conflicts over resources and territories, some pirates may even be seekin' treasures in the remnants of the old world.\n\nSo hoist yer flag, keep a weather eye on the horizon, and enjoy the adventure that awaits! Yarrr!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [35]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client = Client()

coding_prompt = """You are an expert coding assistant. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your answers should be precise, clear, and provide example code if applicable.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""

# Create a ChatPromptTemplate from the string
coding_prompt_template = ChatPromptTemplate.from_template(coding_prompt)

# Push the prompt to LangSmith
client.push_prompt("coding-rag-prompt", object=coding_prompt_template)


'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=cf5f72fb-2f6d-5b80-8c1e-bdb916a4631f'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [36]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=cf5f72fb-2f6d-5b80-8c1e-bdb916a4631f'

In [39]:
import os
from langsmith import Client
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
client = Client(api_key=LANGCHAIN_API_KEY)
prompt1 = client.pull_prompt("chess-llm", include_model=True)

c:\users\ayush singh\appdata\local\programs\python\python39\lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [40]:
prompt1

ChatPromptTemplate(input_variables=['', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'chess-llm', 'lc_hub_commit_hash': '5148fd61967798e001b814f8d507d076752157c42dd56066426898b5cf48903e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[''], input_types={}, partial_variables={}, template='You are a chess related LLM, you answer questions like a {} elo rating chess player.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021412673430>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021412673B50>, root_client=<openai.OpenAI object at 0x0000021412673220>, root_async_client=<openai.AsyncOpenAI object at 0x00000

In [45]:
from openai import OpenAI
from langsmith import Client
from langsmith.client import convert_prompt_to_openai_format

client = Client()
openai_client = OpenAI()

prompt = client.pull_prompt("chess-llm", include_model=False)

variables = {
    "rating": "1400", 
    "question": "What should be the counter move for bishop e4?"  
}

hydrated_prompt = prompt.invoke(variables)

converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages
)

print(response.choices[0].message)


ChatCompletionMessage(content="The move 1...Be4 could be a response to an existing position, but without more context about the overall position on the board or the specific question, it's a bit tricky to recommend a precise counter move. However, generally speaking, you might consider a few different types of responses:\n\n1. If the bishop on e4 is hanging (meaning it can be captured), then you could simply capture it with your knight or another piece if possible.\n2. If the e4 bishop is well-supported and you’re not in a position to capture, you might consider moves that challenge its influence, such as playing f3 (if it’s your bishop) or potentially repositioning your pieces to attack other targets.\n3. If you're looking to create tension in the center, you might consider moves such as d5 if the position allows that.\n\nCould you provide a bit more context, like the full position or the color you are playing so I can give more specific advice?", refusal=None, role='assistant', annot

In [46]:
prompt1

ChatPromptTemplate(input_variables=['', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'chess-llm', 'lc_hub_commit_hash': '5148fd61967798e001b814f8d507d076752157c42dd56066426898b5cf48903e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[''], input_types={}, partial_variables={}, template='You are a chess related LLM, you answer questions like a {} elo rating chess player.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021412673430>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021412673B50>, root_client=<openai.OpenAI object at 0x0000021412673220>, root_async_client=<openai.AsyncOpenAI object at 0x00000

In [49]:
variables = {
    "rating": "1400", 
    "question": "What should be the counter move for bishop e4?"  
}

hydrated_prompt = prompt.invoke(variables)

print(hydrated_prompt)

messages=[SystemMessage(content='You are a chess related LLM, you answer questions like a 1400 elo rating chess player.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What should be the counter move for bishop e4?', additional_kwargs={}, response_metadata={})]


In [50]:
import os
from langsmith import Client
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
client = Client(api_key=LANGCHAIN_API_KEY)
prompt = client.pull_prompt("chess-llm:7820599e", include_model=True)

c:\users\ayush singh\appdata\local\programs\python\python39\lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [51]:
from openai import OpenAI
from langsmith import Client
from langsmith.client import convert_prompt_to_openai_format

client = Client()
openai_client = OpenAI()

prompt = client.pull_prompt("chess-llm", include_model=False)

variables = {
    "question": "What should be the counter move for bishop e4?",
    "rating": "1400" 
}

hydrated_prompt = prompt.invoke(variables)

converted = convert_prompt_to_openai_format(hydrated_prompt)
converted_messages = converted.get("messages", [])

if not converted_messages:
    raise ValueError("No messages found after converting hydrated prompt. Check your prompt or variables.")

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages
)

print(response.choices[0].message)


ChatCompletionMessage(content="To better assist you, could you clarify the context of the position? What is your opponent's setup, and what is your piece placement? Understanding the whole board or the moves that led to Bishop e4 will help me guide you effectively.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [52]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client = Client()

coding_prompt = """You are an expert coding assistant. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your answers should be precise, clear, and provide example code if applicable.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""

coding_prompt_template = ChatPromptTemplate.from_template(coding_prompt)

client.push_prompt("coding-rag-prompt", object=coding_prompt_template)


'https://smith.langchain.com/prompts/coding-rag-prompt/407049c3?organizationId=cf5f72fb-2f6d-5b80-8c1e-bdb916a4631f'

In [54]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client = Client()

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

coding_prompt = """You are an expert coding assistant. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your answers should be precise, clear, and provide example code if applicable.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""

coding_prompt_template = ChatPromptTemplate.from_template(coding_prompt)

chain = coding_prompt_template | model

client.push_prompt("coding-runnable-sequence", object=chain)


'https://smith.langchain.com/prompts/coding-runnable-sequence/24a3b079?organizationId=cf5f72fb-2f6d-5b80-8c1e-bdb916a4631f'